In [6]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
w

In [7]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()



'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/126/chromedriver'

In [12]:
import requests
from selenium.webdriver.common.by import By
import urllib
import time

In [1]:
# prompt: import drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [85]:
!pip install tqdm

In [91]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from tqdm import tqdm
from google.colab import files

# Setup WebDriver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

def scrape_image_src(query, max_docid):
    # Step 1: Open the URL
    search_url = f"https://www.google.com/search?q={query}&tbm=isch"
    driver.get(search_url)
    time.sleep(3)  # Wait for the page to load

    # Step 2: Get all data-docid values
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    data_docid_elements = soup.select('[data-docid]')
    data_docids = [element['data-docid'] for element in data_docid_elements][:max_docid]

    image_sources = []

    # Step 3: Loop over data-docid values
    for data_docid in tqdm(data_docids, desc=f"Processing {query}", leave=False):
        image_url = f"https://www.google.com/search?q={query}&tbm=isch#vhid={data_docid}&vssid=mosaic"
        driver.get(image_url)
        time.sleep(1)  # Wait for the page to load

        # Step 4: Get the src of img element with the class
        try:
            img_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'img.sFlh5c.pT0Scc.iPVvYb'))
            )
            img_element = driver.find_element(By.CSS_SELECTOR, 'img.sFlh5c.pT0Scc.iPVvYb')
            img_src = img_element.get_attribute('src')
            image_sources.append(img_src)
        except Exception as e:
            print(f"Error retrieving image source for data-docid {data_docid}: {e}")

    return image_sources

# Dictionary of diseases and conditions with max docid values
diseases = {
    # 'acne': {
    #     'disease+close': 80,
    #     'close+up': 80,
    #     'face+close': 80
    # },
    # 'melanoma': {
    #     'arm+close': 50,
    #     'leg+close': 50,
    #     'disease+close': 50,
    #     'close+up': 50,
    #     'face+close': 50
    # },
    # 'eczema': {
    #     'arm+close': 50,
    #     'leg+close': 50,
    #     'disease+close': 50,
    #     'close+up': 50,
    #     'face+close': 50
    # },
    'herpes': {
        'arm+close': 50,
        'leg+close': 50,
        'disease+close': 50,
        'close+up': 50,
        'face+close': 50
    },
    'lupus': {
        'arm+close': 50,
        'leg+close': 50,
        'disease+close': 50,
        'close+up': 50,
        'face+close': 50
    },
    'tinea+ringworm': {
        'arm+close': 50,
        'leg+close': 50,
        'disease+close': 50,
        'close+up': 50,
        'face+close': 50
    },
    'urticaria+hives': {
        'arm+close': 50,
        'leg+close': 50,
        'disease+close': 50,
        'close+up': 50,
        'face+close': 50
    },
    'warts': {
        'arm+close': 60,
        'leg+close': 60,
        'disease+close': 60,
        'close+up': 60,
    }
}

for disease, conditions in diseases.items():
    csv_file = f"{disease}_image_sources.csv"
    csv_columns = ['disease', 'link']

    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=csv_columns)
        writer.writeheader()

        for condition, max_docid in conditions.items():
            query = f"{disease}+{condition}"
            print(f"Scraping images for query: {query}")
            image_sources = scrape_image_src(query, max_docid)

            for src in image_sources:
                writer.writerow({'disease': disease, 'link': src})

    files.download(csv_file)
    print(f"Scraping complete for {disease}. Saving and downloading {csv_file}...")


# Close the WebDriver
driver.quit()

print("Scraping complete for all diseases.")


Scraping images for query: herpes+arm+close


Processing herpes+arm+close:  76%|███████▌  | 38/50 [01:52<01:27,  7.31s/it]

Error retrieving image source for data-docid PlQePBzZP4MCMM: Message: 
Stacktrace:
#0 0x58360fb186aa <unknown>
#1 0x58360f7fb0dc <unknown>
#2 0x58360f847931 <unknown>
#3 0x58360f847a21 <unknown>
#4 0x58360f88c234 <unknown>
#5 0x58360f86a89d <unknown>
#6 0x58360f8895c3 <unknown>
#7 0x58360f86a613 <unknown>
#8 0x58360f83a4f7 <unknown>
#9 0x58360f83ae4e <unknown>
#10 0x58360fade87b <unknown>
#11 0x58360fae2921 <unknown>
#12 0x58360faca36e <unknown>
#13 0x58360fae3482 <unknown>
#14 0x58360faaeccf <unknown>
#15 0x58360fb080a8 <unknown>
#16 0x58360fb08280 <unknown>
#17 0x58360fb177dc <unknown>
#18 0x7983a6223ac3 <unknown>



Scraping images for query: herpes+leg+close


Processing herpes+leg+close:  40%|████      | 20/50 [00:58<02:39,  5.31s/it]

...

Processing warts+close+up:  80%|████████  | 48/60 [02:15<01:04,  5.41s/it]

Error retrieving image source for data-docid xkRaMPOaUYE_bM: Message: 
Stacktrace:
#0 0x58360fb186aa <unknown>
#1 0x58360f7fb0dc <unknown>
#2 0x58360f847931 <unknown>
#3 0x58360f847a21 <unknown>
#4 0x58360f88c234 <unknown>
#5 0x58360f86a89d <unknown>
#6 0x58360f8895c3 <unknown>
#7 0x58360f86a613 <unknown>
#8 0x58360f83a4f7 <unknown>
#9 0x58360f83ae4e <unknown>
#10 0x58360fade87b <unknown>
#11 0x58360fae2921 <unknown>
#12 0x58360faca36e <unknown>
#13 0x58360fae3482 <unknown>
#14 0x58360faaeccf <unknown>
#15 0x58360fb080a8 <unknown>
#16 0x58360fb08280 <unknown>
#17 0x58360fb177dc <unknown>
#18 0x7983a6223ac3 <unknown>



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Scraping complete for warts. Saving and downloading warts_image_sources.csv...
Scraping complete for all diseases.


In [96]:
import os
import requests
import pandas as pd
from PIL import Image
from io import BytesIO
from tqdm import tqdm

# Function to check if an image is corrupted and in JPEG format
def is_valid_jpeg(img_data):
    try:
        img = Image.open(BytesIO(img_data))
        img.verify()  # Verify if the image is corrupted
        if img.format not in ['JPEG', 'JPG']:
            return False
        return True
    except (IOError, SyntaxError) as e:
        return False

# List of diseases
diseases = [
    'acne',
    'melanoma',
    'eczema',
    'herpes',
    'lupus',
    'tinea+ringworm',
    'urticaria+hives',
    'warts'
]

# Create folders and download images
for disease in tqdm(diseases, desc="Processing diseases"):
    # Create directory for the disease
    if not os.path.exists(disease):
        os.makedirs(disease)

    # Read the corresponding CSV file
    csv_file = f"{disease}_image_sources.csv"
    if not os.path.exists(csv_file):
        print(f"CSV file for {disease} not found, skipping...")
        continue

    df = pd.read_csv(csv_file)

    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Downloading images for {disease}", leave=False):
        image_url = row['link']
        try:
            response = requests.get(image_url, timeout=10)
            if response.status_code == 200:
                img_data = response.content
                if is_valid_jpeg(img_data):
                    # Save the image
                    folder_path = os.path.join("/content/drive/MyDrive/dataset/scrape", disease)
                    img_path = os.path.join(folder_path, f"{disease}_{idx}.jpg")
                    os.makedirs(folder_path, exist_ok=True)  # Ensure disease folder exists
                    with open(img_path, 'wb') as img_file:
                        img_file.write(img_data)
                    print(f"Image {img_path} downloaded successfully.")
                else:
                    print(f"Image {image_url} is not a valid JPEG, skipping...")
            else:
                print(f"Failed to download {image_url}, status code: {response.status_code}")
        except Exception as e:
            print(f"Error downloading {image_url}: {e}")

print("Processing complete.")


Processing diseases:   0%|          | 0/8 [00:00<?, ?it/s]


Image /content/drive/MyDrive/dataset/scrape/acne/acne_0.jpg downloaded successfully.
Image /content/drive/MyDrive/dataset/scrape/acne/acne_1.jpg downloaded successfully.


Image /content/drive/MyDrive/dataset/scrape/acne/acne_2.jpg downloaded successfully.


Image /content/drive/MyDrive/dataset/scrape/acne/acne_3.jpg downloaded successfully.


Image /content/drive/MyDrive/dataset/scrape/acne/acne_4.jpg downloaded successfully.
Image /content/drive/MyDrive/dataset/scrape/acne/acne_5.jpg downloaded successfully.


...

Image /content/drive/MyDrive/dataset/scrape/warts/warts_229.jpg downloaded successfully.


Failed to download https://hudsondermatology.com/wp-content/uploads/2024/03/shutterstock_1943240353-scaled.jpg, status code: 403


Image /content/drive/MyDrive/dataset/scrape/warts/warts_231.jpg downloaded successfully.



Processing diseases: 100%|██████████| 8/8 [11:52<00:00, 89.11s/it]

Image /content/drive/MyDrive/dataset/scrape/warts/warts_232.jpg downloaded successfully.
Processing complete.


In [2]:
import os
import random
import shutil
from tqdm import tqdm

# Function to shuffle and split images into training, validation, and testing sets
def shuffle_and_split_images(disease_folder, train_folder, val_folder, test_folder):
    # Get the list of image filenames
    images = os.listdir(disease_folder)
    random.shuffle(images)

    # Calculate number of images for each set based on the ratio 70:20:10
    num_images = len(images)
    num_train = int(0.7 * num_images)
    num_val = int(0.2 * num_images)
    num_test = num_images - num_train - num_val

    # Move images to corresponding folders
    for i, image in enumerate(tqdm(images, desc=f"Splitting images {disease_folder}", leave=False)):
        src = os.path.join(disease_folder, image)
        if i < num_train:
            dest = os.path.join(train_folder, image)
        elif i < num_train + num_val:
            dest = os.path.join(val_folder, image)
        else:
            dest = os.path.join(test_folder, image)
        shutil.move(src, dest)

# List of diseases
diseases = [
    'Acne',
    'Melanoma',
    'Eczema',
    'Herpes',
    'Lupus',
    'Tinea Ringworm',
    'Urticaria Hives',
    'Warts'
]

# Create Training, Validation, and Testing folders
train_folder = os.path.join("/content/drive/MyDrive/dataset/scrape", "Training")
val_folder = os.path.join("/content/drive/MyDrive/dataset/scrape", "Validation")
test_folder = os.path.join("/content/drive/MyDrive/dataset/scrape", "Testing")
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Create disease folders in Training, Validation, and Testing folders
for disease in diseases:
    train_disease_folder = os.path.join(train_folder, disease)
    val_disease_folder = os.path.join(val_folder, disease)
    test_disease_folder = os.path.join(test_folder, disease)
    os.makedirs(train_disease_folder, exist_ok=True)
    os.makedirs(val_disease_folder, exist_ok=True)
    os.makedirs(test_disease_folder, exist_ok=True)

# Iterate through each disease folder and split the images
for disease in diseases:
    disease_folder = os.path.join("/content/drive/MyDrive/dataset/scrape", disease)
    if not os.path.exists(disease_folder):
        print(f"Folder {disease} not found, skipping...")
        continue

    shuffle_and_split_images(disease_folder,
                              os.path.join(train_folder, disease),
                              os.path.join(val_folder, disease),
                              os.path.join(test_folder, disease))

print("Splitting images complete.")


Splitting images complete.


In [5]:
import os
import shutil

def copy_healthy_folders(src_dataset, dest_dataset):
    # Iterate through the directories in the source dataset
    for root, dirs, files in os.walk(src_dataset):
        for dir_name in dirs:
            # Check if the directory is named 'Healthy'
            if dir_name == 'Healthy':
                src_healthy_dir = os.path.join(root, dir_name)
                dest_healthy_dir = src_healthy_dir.replace(src_dataset, dest_dataset)
                # Copy 'Healthy' directory from source to destination dataset
                if os.path.exists(dest_healthy_dir):
                    shutil.rmtree(dest_healthy_dir)
                shutil.copytree(src_healthy_dir, dest_healthy_dir)

# Source and destination datasets
src_dataset = '/content/drive/MyDrive/dataset/Penyakit Kulit'
dest_dataset = '/content/drive/MyDrive/dataset/scrape'

# Copy 'Healthy' folders from source to destination dataset
copy_healthy_folders(os.path.join(src_dataset, 'Train'), os.path.join(dest_dataset, 'Train'))
copy_healthy_folders(os.path.join(src_dataset, 'Test'), os.path.join(dest_dataset, 'Test'))
copy_healthy_folders(os.path.join(src_dataset, 'Validation'), os.path.join(dest_dataset, 'Validation'))

print("Copying 'Healthy' folders complete.")


Copying 'Healthy' folders complete.
